In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
%pwd

'/Users/ryanmaki'

In [5]:
bank_bal=pd.read_csv('OBA_410_Group_Data.csv')
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,0,0.000000,0.006923,0.000000,0.593501,0.593551,0.998775,0.795533,0.807547,0.300687,...,0.224792,2.090270e-04,0.623797,0.593502,0.770902,0.285511,0.026790,0.565152,1,0.019171
1,1,0.024277,0.000000,0.033514,0.591324,0.591324,0.994015,0.791503,0.803775,0.303602,...,0.584583,2.710000e+09,0.623818,0.591322,0.827165,0.276846,0.026791,0.565158,1,0.054959
2,1,0.066933,0.057185,0.054821,0.601861,0.601861,0.998825,0.796779,0.808717,0.302760,...,0.525651,5.803037e-03,0.623648,0.601857,1.000000,0.182790,0.026763,0.565021,1,0.009178
3,0,0.081412,0.091256,0.091761,0.602935,0.602935,0.973875,0.767759,0.781429,0.304248,...,0.412621,1.920000e+09,0.623783,0.602932,0.813135,0.278182,0.026791,0.565158,1,0.037886
4,0,0.082826,0.069287,0.088388,0.646846,0.646846,0.997083,0.795795,0.807815,0.304684,...,0.000000,0.000000e+00,0.623730,0.646841,0.748252,0.284685,0.026791,0.565158,1,0.019908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6814,0,0.787988,0.804514,0.783928,0.608015,0.608015,0.999147,0.798222,0.809903,0.304606,...,0.905790,2.948022e-03,0.624791,0.608017,0.847232,0.277502,0.026791,0.565159,1,0.044549
6815,0,0.818018,0.801570,0.814551,0.634983,0.634983,0.999278,0.797758,0.809581,0.303522,...,0.923155,1.093630e-04,0.623786,0.634983,0.848965,0.278178,0.026794,0.565170,1,0.037918
6816,0,0.864964,0.942706,0.932598,0.622371,0.622371,0.999308,0.797793,0.809673,0.303519,...,0.959320,2.271900e-04,0.624001,0.622370,0.849236,0.276178,0.026791,0.565158,1,0.074893
6817,0,0.971530,1.000000,1.000000,0.640035,0.640035,0.999571,0.798105,0.809906,0.303516,...,0.981315,1.534960e-04,0.624305,0.640035,0.851547,0.277013,0.026799,0.565196,1,0.051752


# Data Sampling

In [6]:
bank_bal=bank_bal.groupby('Bankrupt?',group_keys=False).apply(lambda x: x.sample(1000,replace=True,random_state=0))
bank_bal['Bankrupt?'].value_counts()

0    1000
1    1000
Name: Bankrupt?, dtype: int64

In [7]:
bank_bal

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2935,0,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,...,0.804195,5.797642e-03,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
2807,0,0.494028,0.556149,0.539697,0.603273,0.603021,0.999016,0.797448,0.809353,0.303527,...,0.806766,1.678171e-03,0.623481,0.603273,0.841082,0.281947,0.027068,0.566204,1,0.023555
1834,0,0.478672,0.541976,0.531881,0.605226,0.605226,0.999008,0.797419,0.809341,0.303494,...,0.801978,4.771583e-03,0.623800,0.605225,0.840553,0.280316,0.026973,0.565898,1,0.027411
3475,0,0.503924,0.551297,0.549280,0.607064,0.607064,0.998987,0.797522,0.809440,0.303718,...,0.804269,4.786887e-02,0.624181,0.607060,0.840859,0.282237,0.027089,0.566266,1,0.023043
5150,0,0.536635,0.597580,0.580813,0.610466,0.610141,0.999138,0.797629,0.809496,0.303588,...,0.832874,3.267898e-03,0.625247,0.610468,0.842125,0.276202,0.026792,0.565160,1,0.073883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,1,0.454346,0.513792,0.506023,0.597522,0.597522,0.998899,0.797237,0.809064,0.303405,...,0.771996,6.000143e-03,0.621979,0.597522,0.837544,0.283088,0.026463,0.562888,1,0.021747
1791,1,0.477990,0.538323,0.529632,0.616966,0.616930,0.999190,0.797473,0.809361,0.303207,...,0.797931,2.171998e-02,0.619427,0.616962,0.840159,0.276686,0.027789,0.567605,1,0.058527
368,1,0.417540,0.470726,0.460892,0.600852,0.600852,0.998935,0.797160,0.809086,0.303195,...,0.754696,8.433273e-03,0.624491,0.600848,0.836693,0.279759,0.026550,0.563668,1,0.029296
1250,1,0.466728,0.490078,0.501097,0.607850,0.607850,0.998855,0.797196,0.809109,0.303425,...,0.767057,8.094303e-03,0.623321,0.607845,0.834982,0.293298,0.026485,0.563103,1,0.015432


# Logistic Regression¶

In [8]:
bank_bal[bank_bal['Bankrupt?']==1].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1.000000e+03,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,1.0,0.42,0.46,0.46,0.60,0.60,1.00,0.80,0.81,0.30,...,0.74,5.630000e+07,0.62,0.60,0.83,0.29,0.03,0.56,1.0,0.02
std,0.0,0.08,0.11,0.09,0.01,0.01,0.00,0.00,0.00,0.01,...,0.08,5.971907e+08,0.01,0.01,0.06,0.05,0.02,0.01,0.0,0.05
min,1.0,0.02,0.00,0.03,0.53,0.53,0.99,0.76,0.76,0.24,...,0.41,0.000000e+00,0.59,0.53,0.00,0.00,0.02,0.55,1.0,0.00
25%,1.0,0.39,0.43,0.44,0.59,0.59,1.00,0.80,0.81,0.30,...,0.72,0.000000e+00,0.62,0.59,0.83,0.28,0.03,0.56,1.0,0.02
50%,1.0,0.44,0.49,0.49,0.60,0.60,1.00,0.80,0.81,0.30,...,0.77,0.000000e+00,0.62,0.60,0.84,0.29,0.03,0.56,1.0,0.02
75%,1.0,0.47,0.53,0.52,0.60,0.60,1.00,0.80,0.81,0.30,...,0.79,1.000000e-02,0.62,0.60,0.84,0.29,0.03,0.57,1.0,0.02
max,1.0,0.58,0.58,0.62,0.67,0.67,1.00,0.80,0.81,0.31,...,0.82,9.170000e+09,0.68,0.67,1.00,1.00,0.26,0.62,1.0,0.80


In [9]:
bank_bal[bank_bal['Bankrupt?']==0].describe().round(2)

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
count,1000.0,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,...,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,0.0,0.51,0.56,0.56,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.05
std,0.0,0.06,0.06,0.06,0.01,0.01,0.01,0.02,0.03,0.01,...,0.04,0.04,0.01,0.01,0.00,0.01,0.02,0.01,0.0,0.06
min,0.0,0.00,0.01,0.00,0.45,0.45,0.61,0.16,0.00,0.00,...,0.22,0.00,0.55,0.45,0.77,0.27,0.00,0.33,1.0,0.01
25%,0.0,0.48,0.54,0.53,0.60,0.60,1.00,0.80,0.81,0.30,...,0.80,0.00,0.62,0.60,0.84,0.28,0.03,0.57,1.0,0.03
50%,0.0,0.50,0.56,0.55,0.61,0.61,1.00,0.80,0.81,0.30,...,0.81,0.00,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.04
75%,0.0,0.54,0.59,0.59,0.61,0.61,1.00,0.80,0.81,0.30,...,0.83,0.01,0.62,0.61,0.84,0.28,0.03,0.57,1.0,0.06
max,0.0,1.00,0.95,0.96,0.66,0.66,1.00,0.81,0.82,0.33,...,0.94,0.54,0.70,0.66,0.85,0.33,0.54,0.74,1.0,1.00


## controlling the complexity of log regression, using parameter 'c'

In [10]:
X_bank, y_bank=bank_bal.iloc[:,1:], bank_bal['Bankrupt?'] 
display(X_bank.head(2))
display(y_bank.head(2))

,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,Continuous interest rate (after tax),...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
2935,0.495686,0.550916,0.542963,0.599209,0.599209,0.999001,0.797404,0.809320,0.303483,0.781574,...,0.804195,0.005798,0.623865,0.599205,0.840688,0.279388,0.027243,0.566668,1,0.030801
2807,0.494028,0.556149,0.539697,0.603273,0.603021,0.999016,0.797448,0.809353,0.303527,0.781613,...,0.806766,0.001678,0.623481,0.603273,0.841082,0.281947,0.027068,0.566204,1,0.023555


2935    0
2807    0
Name: Bankrupt?, dtype: int64

## Feature Selection

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dt=DecisionTreeClassifier(random_state=0)
dt.fit(X_bank,y_bank)

DecisionTreeClassifier(random_state=0)

In [13]:
sum(dt.feature_importances_>.01)

16

In [14]:
X_bank_sel=X_bank[X_bank.columns[dt.feature_importances_>.01]]
X_bank_sel.head()

,Continuous interest rate (after tax),Operating Expense Rate,Interest-bearing debt interest rate,Net Value Per Share (A),Persistent EPS in the Last Four Seasons,Operating Profit Growth Rate,Total Asset Growth Rate,Quick Ratio,Interest Expense Ratio,Total debt/Total net worth,Accounts Receivable Turnover,Operating profit per person,Allocation rate per person,Cash Turnover Rate,Cash Flow to Sales,Interest Coverage Ratio (Interest expense to EBIT)
2935,0.781574,5.820000e+09,0.000672,0.177447,0.217831,0.848084,5.550000e+09,0.007517,0.632188,0.006354,0.001469,0.396373,0.005721,2.580000e+08,0.671572,0.566668
2807,0.781613,1.038420e-04,0.000359,0.190342,0.224355,0.848025,8.000000e+09,0.003586,0.631575,0.009883,0.001108,0.398295,0.006320,6.650000e+09,0.671579,0.566204
1834,0.781601,1.474370e-04,0.000141,0.171421,0.218020,0.848012,5.830000e+09,0.006185,0.631163,0.007635,0.000363,0.394677,0.015585,1.250000e+09,0.671575,0.565898
3475,0.781594,1.973910e-04,0.000484,0.189415,0.216129,0.848184,1.039850e-04,0.004459,0.632241,0.010284,0.002682,0.392879,0.048975,2.072150e-04,0.671454,0.566266
5150,0.781749,9.560000e+09,0.000000,0.234419,0.245722,0.848215,7.490000e+09,0.022148,0.630614,0.001962,0.001175,0.403247,0.002276,2.695690e-04,0.671648,0.565160


## transform the features

In [15]:
scaler1=MinMaxScaler()
X_bank_trns=scaler1.fit_transform(X_bank_sel)
X_bank_trns

array([[9.78771560e-01, 5.87285570e-01, 7.00069792e-13, ...,
        2.58258258e-02, 9.72108922e-01, 5.79849651e-01],
       [9.78829637e-01, 3.85065590e-16, 3.73995833e-13, ...,
        6.65665666e-01, 9.72118950e-01, 5.78705083e-01],
       [9.78812680e-01, 4.78415742e-15, 1.46889583e-13, ...,
        1.25125125e-01, 9.72112531e-01, 5.77951033e-01],
       ...,
       [9.78397658e-01, 5.07578204e-15, 8.43834375e-13, ...,
        1.39725726e-14, 9.72129663e-01, 5.72450276e-01],
       [9.78439202e-01, 2.44808274e-14, 4.43794792e-13, ...,
        9.09909910e-02, 9.72104756e-01, 5.71057391e-01],
       [9.69751507e-01, 4.86501715e-13, 0.00000000e+00, ...,
        2.34577578e-14, 9.72031743e-01, 5.76126590e-01]])

## validation curve functions

In [16]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_trns, y_bank, test_size=0.2, random_state=0)

In [17]:
train_scores_log_reg, test_scores_log_reg=validation_curve(LogisticRegression(penalty='l1', solver='saga', max_iter=100000), X_train, y_train, param_name='C',
                                           param_range=[0.01,0.05,1,5,10,20], cv=4)

In [18]:
train_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.57  , 0.8023, 0.8071, 0.8088, 0.8106])

In [19]:
test_scores_log_reg.mean(axis=1).round(4)

array([0.5   , 0.5581, 0.7981, 0.8019, 0.8038, 0.8062])

In [20]:
log_reg=LogisticRegression(max_iter=1000, C=100)
log_reg.fit(X_train, y_train)

LogisticRegression(C=100, max_iter=1000)

In [21]:
print('log reg acc on train: {:.2%}'.format(log_reg.score(X_train, y_train)))
print('log reg acc on test: {:.2%}'.format(log_reg.score(X_test, y_test)))

log reg acc on train: 80.12%
log reg acc on test: 79.00%


In [22]:
log_reg.coef_

array([[ 5.36632142e+00,  7.27259627e-01, -2.35231972e+00,
        -1.61458275e+01, -5.71581688e+01, -3.39453542e+00,
         5.70044131e-02,  6.42682340e-01,  2.96433513e+00,
         3.08322169e+00,  2.63528966e+00,  3.95781066e+00,
         1.19253712e+00, -1.14549134e-01, -1.53656702e-01,
         1.15076150e+00]])

In [23]:
p1=[0.2,0.4,0.5,0.1,0.4,0.3,0.2,0.5,0.2,0.5,0.5,0.2,0.2,0.4,0.5,0.2,0.1]
log_reg.predict([p1])

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [24]:
import numpy as np
np.set_printoptions(suppress=True)

In [25]:
log_reg.predict_proba([p1]).round(4)

ValueError: X has 17 features, but LogisticRegression is expecting 16 features as input.

In [26]:
log_reg.classes_

array([0, 1])

In [184]:
log_reg.coef_.round(4)

array([[ -7.5535,  18.9253, -14.9979,  -0.0461, -12.9969,   1.3364,
          3.1745,   5.976 ,  -1.7221,   3.8634, -11.7612,   0.5294,
        -15.1517,  -0.1951,   0.0368,  -2.0263, -26.5054]])

In [185]:
log_reg.intercept_.round(4)

array([16.6968])

# SVM 

In [186]:
X_bank_trns

array([[0.77331281, 0.9825482 , 0.9164898 , ..., 0.68664439, 0.        ,
        0.03250087],
       [0.89523873, 0.98132336, 0.9158897 , ..., 0.95389559, 0.        ,
        0.07486554],
       [0.72165155, 0.97649448, 0.91216798, ..., 0.59529901, 0.        ,
        0.02562554],
       ...,
       [0.66978954, 0.97090779, 0.90653586, ..., 0.68419341, 0.        ,
        0.01565286],
       [0.78930639, 0.97678527, 0.91286168, ..., 0.7690724 , 0.        ,
        0.04668739],
       [0.70428614, 0.97405802, 0.90008616, ..., 0.65938053, 0.        ,
        0.01333904]])

In [187]:
svm1=LinearSVC(random_state=0)
svm1.fit(X_train,y_train)

LinearSVC(random_state=0)

In [188]:
svm1.coef_

array([[-2.3922309 ,  3.02109436, -1.27422496, -0.00527948, -2.20330322,
        -0.        ,  1.20639627,  3.02475093, -0.16313754,  0.89815626,
        -1.72630618,  0.0456347 , -5.17381964, -0.04322548, -0.00509823,
        -0.56900942, -3.13605435]])

In [189]:
svm1.intercept_

array([3.50851562])

In [190]:
print('svm acc on train: {:.2%}'.format(svm1.score(X_train, y_train)))
print('svm acc on test: {:.2%}'.format(svm1.score(X_test, y_test)))

svm acc on train: 81.94%
svm acc on test: 84.25%


In [191]:
b=[[0.616,0.999,0.781,0.00001,0.4719,0.0005,0.6306,0.3696,0.0415,0.0022,0.2770,0.9532,0,1.1,0.7021,0.0029,0.0345]]

In [192]:
b_trns=scaler1.transform(b)
b_trns

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[ 0.77180473,  0.97523232,  0.9003833 , -0.        ,  0.46878615,
         0.        ,  0.28705302,  0.3696    ,  0.08202934,  0.        ,
         0.14518228,  0.        , -2.69984748,  0.        ,  0.68655891,
         0.        ,  0.0325461 ]])

In [193]:
svm1.predict(b_trns)

array([1])

In [194]:
svm1.intercept_.round(2)

array([3.51])

In [195]:
svm1.coef_.round(2)

array([[-2.39,  3.02, -1.27, -0.01, -2.2 , -0.  ,  1.21,  3.02, -0.16,
         0.9 , -1.73,  0.05, -5.17, -0.04, -0.01, -0.57, -3.14]])

## controlling the complexity of svm by change parameter 'C'

In [234]:
LinearSVC()

LinearSVC()

In [235]:
C_range=[.1,1,5,10,50,100]

In [236]:
train_scores_SVM, test_scores_SVM=validation_curve(LinearSVC(random_state=0, max_iter=100000), X_bank_trns, y_bank, param_name='C',
                                           param_range=[.1,1,5,10,50,100], cv=4)

In [237]:
train_scores_SVM.round(4)

array([[0.7927, 0.818 , 0.786 , 0.79  ],
       [0.8233, 0.8247, 0.8193, 0.8167],
       [0.826 , 0.8273, 0.8293, 0.822 ],
       [0.83  , 0.826 , 0.8267, 0.8233],
       [0.8307, 0.8287, 0.8247, 0.8247],
       [0.8293, 0.8287, 0.8227, 0.8227]])

In [238]:
print('ave cross in 4 folds for different C values:')
print('training:', train_scores_SVM.mean(axis=1).round(4))
print('test', test_scores_SVM.mean(axis=1).round(4))

ave cross in 4 folds for different C values:
training: [0.7967 0.821  0.8262 0.8265 0.8272 0.8258]
test [0.797  0.8205 0.8225 0.8215 0.824  0.823 ]


In [240]:
train_scores_SVM.mean()

0.8205555555555555

# Decision tree

In [202]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel, y_bank, test_size=0.2, random_state=0)

In [203]:
X_train

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
1270,0.607698,0.999075,0.781581,1.204932e-04,0.465501,0.000447,0.634492,0.392679,0.037097,0.035477,0.276596,0.004880,0.917185,4.260000e+09,0.727656,0.006280,0.016797
6201,0.591022,0.998626,0.781222,3.020425e-04,0.478040,0.000429,0.629540,0.374118,0.012903,0.035610,0.276956,0.000185,0.943350,2.785699e-04,0.542416,0.000875,0.043125
1639,0.590856,0.998732,0.781208,1.927660e-04,0.458967,0.000388,0.629974,0.379328,0.024355,0.013620,0.276177,0.002407,0.917745,2.610000e+09,0.770937,0.012318,0.022717
3872,0.602524,0.999032,0.781622,7.990000e+08,0.484796,0.000414,0.633260,0.379072,0.023065,0.020758,0.277173,0.006388,0.943059,2.593929e-04,0.305700,0.009755,0.025943
5443,0.603511,0.999023,0.781639,1.045354e-04,0.469698,0.000438,0.630630,0.369637,0.026452,0.003649,0.277012,0.001440,0.938743,2.590000e+09,0.806450,0.001010,0.046239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4916,0.601183,0.999023,0.781632,6.760000e+09,0.460872,0.000424,0.630873,0.374808,0.036129,0.006172,0.277277,0.003578,0.945173,1.620000e+09,0.737087,0.004855,0.024575
6411,0.600989,0.998889,0.781362,1.984322e-04,0.458103,0.000367,0.630012,0.391831,0.049032,0.006413,0.278349,0.004280,0.919304,1.160000e+09,0.833007,0.004843,0.015417
4491,0.616930,0.999190,0.781618,1.525245e-04,0.463412,0.000484,0.633105,0.372200,0.012258,0.042561,0.277020,0.000580,0.932301,2.040000e+08,0.867257,0.021720,0.058527
3316,0.625168,0.999191,0.781735,2.840032e-04,0.483070,0.000465,0.630703,0.369637,0.018710,0.008820,0.277028,0.009467,0.950731,2.451023e-04,0.923759,0.020385,0.133604


In [204]:
train_scores_decision_tree, test_scores_decision_tree=validation_curve(DecisionTreeClassifier(random_state=0), X_train, y_train, param_name='max_leaf_nodes', 
                                          param_range=[2,6,8,10,15,25,50], cv=5)

In [205]:
dt0=DecisionTreeClassifier(random_state=0, max_leaf_nodes=10)
dt0.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [206]:
print('dt0 acc on train: {:.2%}'.format(dt0.score(X_train, y_train)))
print('dt0 acc on test: {:.2%}'.format(dt0.score(X_test, y_test)))

dt0 acc on train: 90.19%
dt0 acc on test: 89.50%


In [207]:
p1=[0.616,0.999,0.781,0.0001,0.471,0.0005,0.631,0.3696,0.0415,0.0022,0.27698,0,0.953,0,0.7022,0.0029,0.0345]
dt0.predict([p1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])

## visulalzing our decision tree

In [208]:
export_graphviz(dt0, out_file='dt0_vis.dot', feature_names=X_bank_sel.columns, class_names=dt0.classes_.astype(str), impurity=False, filled=True)

In [209]:
dt0.classes_

array([0, 1])

## Feature Importance 

In [210]:
dt0.feature_importances_.round(3)

array([0.027, 0.043, 0.567, 0.   , 0.   , 0.049, 0.124, 0.   , 0.024,
       0.   , 0.   , 0.027, 0.   , 0.027, 0.   , 0.   , 0.111])

In [211]:
X_bank_sel.columns

Index([' Realized Sales Gross Margin', ' Operating Profit Rate',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Cash flow rate', ' Net Value Growth Rate', ' Interest Expense Ratio',
       ' Borrowing dependency', ' Net Worth Turnover Rate (times)',
       ' Allocation rate per person', ' Inventory/Working Capital',
       ' Inventory/Current Liability', ' Retained Earnings to Total Assets',
       ' Cash Turnover Rate', ' Current Liability to Liability',
       ' Total assets to GNP price', ' Equity to Liability'],
      dtype='object')

In [212]:
feat_imp=pd.DataFrame(data=dt0.feature_importances_.round(3),index=X_bank_sel.columns, columns=['importance'])
feat_imp.sort_values('importance', ascending=False)

,importance
Continuous interest rate (after tax),0.567
Interest Expense Ratio,0.124
Equity to Liability,0.111
Net Value Growth Rate,0.049
Operating Profit Rate,0.043
Cash Turnover Rate,0.027
Inventory/Current Liability,0.027
Realized Sales Gross Margin,0.027
Net Worth Turnover Rate (times),0.024
Borrowing dependency,0.000


# Random Forest

In [213]:
bank_rf=RandomForestClassifier(n_estimators=300, random_state=0)
bank_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [214]:
print('bank_rf on train: {:.2%}'.format(bank_rf.score(X_train, y_train)))
print('bank_rf on test: {:.2%}'.format(bank_rf.score(X_test, y_test)))

bank_rf on train: 100.00%
bank_rf on test: 96.50%


## Feature Importance

In [215]:
bank_rf.feature_importances_

array([0.03381974, 0.04262034, 0.13899968, 0.01959498, 0.03919503,
       0.08207163, 0.0847119 , 0.12232535, 0.02289402, 0.03238687,
       0.03639771, 0.02309779, 0.13419101, 0.02712801, 0.01970783,
       0.02654675, 0.11431137])

In [216]:
bank_rf_feat_imp=pd.DataFrame(data=bank_rf.feature_importances_, index=X_bank_sel.columns, columns=['importance'])
bank_rf_feat_imp.sort_values(by=['importance'], ascending=False).round(3)

,importance
Continuous interest rate (after tax),0.139
Retained Earnings to Total Assets,0.134
Borrowing dependency,0.122
Equity to Liability,0.114
Interest Expense Ratio,0.085
Net Value Growth Rate,0.082
Operating Profit Rate,0.043
Cash flow rate,0.039
Inventory/Working Capital,0.036
Realized Sales Gross Margin,0.034


In [217]:
X_test.iloc[20:25,:]

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
1176,0.595007,0.998909,0.781286,8.270000e+09,0.461918,0.000404,0.628257,0.384412,0.023548,0.086678,0.278969,0.015562,0.923034,2.430000e+09,0.707643,0.013448,0.020525
948,0.610595,0.998977,0.781531,2.649232e-04,0.461711,0.000441,0.624179,0.379319,0.037258,0.004766,0.277887,0.006438,0.932147,4.840000e+09,0.909853,0.001133,0.020014
4446,0.611936,0.999137,0.781773,1.265063e-04,0.457142,0.000814,0.630616,0.378273,0.105323,0.005009,0.274562,0.002208,0.951260,3.260000e+09,0.988181,0.000884,0.016198
1756,0.623013,0.998633,0.781034,8.084147e-04,0.486569,0.000390,0.630274,0.373629,0.019355,0.000385,0.277069,0.003182,0.892959,9.330000e+07,0.486479,0.000440,0.031375
1756,0.623013,0.998633,0.781034,8.084147e-04,0.486569,0.000390,0.630274,0.373629,0.019355,0.000385,0.277069,0.003182,0.892959,9.330000e+07,0.486479,0.000440,0.031375


In [218]:
bank_rf.predict(X_test.iloc[20:25,:])

array([1, 1, 0, 1, 1])

# KNN Classifier

In [219]:
train_scores_KNN, test_scores_KNN=validation_curve(KNeighborsClassifier(), X_bank_trns, y_bank, param_name='n_neighbors', 
                                           param_range=[1,2,3,4,5,6], cv=4)

In [220]:
train_scores_KNN.mean(axis=1)

array([1.        , 0.99      , 0.93683333, 0.91983333, 0.88983333,
       0.874     ])

In [221]:
train_scores_KNN.mean()

0.9350833333333334

In [222]:
test_scores_KNN.mean()

0.86775

In [223]:
X_train, X_test, y_train, y_test=train_test_split(X_bank_sel,y_bank,
                                                  random_state=0)

In [224]:
knn_cls_bank_bal=KNeighborsClassifier(n_neighbors=5)

In [225]:
knn_cls_bank_bal.fit(X_train,y_train)

KNeighborsClassifier()

In [226]:
print('knn_cls_bank_bal acc on train: {:.2%}'.format(knn_cls_bank_bal.score(X_train,y_train)))
print('knn_cls_bank_bal acc on test: {:.2%}'.format(knn_cls_bank_bal.score(X_test,y_test)),'\n')

knn_cls_bank_bal acc on train: 83.40%
knn_cls_bank_bal acc on test: 78.60% 



In [227]:
knn_cls_bank_bal.score(X_bank_sel,y_bank)

0.822

In [228]:
w1=[1,1,0.5,0.4,0.3,0.3,0.6,0.7,1,0.4,0.2,0.5,0.7,0.4,0.1,0.9,0.6]
knn_cls_bank_bal.predict([w1])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1])

In [229]:
knn_cls_bank_bal.predict(X_bank_sel)

array([1, 0, 0, ..., 1, 1, 1])

In [230]:
len(knn_cls_bank_bal.predict(X_bank_sel))

2000

In [231]:
predict_array = knn_cls_bank_bal.predict(X_bank_sel)
count_non_zero = 0
for i in predict_array:
    if i != 0:
        count_non_zero += 1
print(count_non_zero)      

1198


In [232]:
sum(knn_cls_bank_bal.predict(X_bank_sel))

1198

In [233]:
X_bank_sel.describe()

,Realized Sales Gross Margin,Operating Profit Rate,Continuous interest rate (after tax),Operating Expense Rate,Cash flow rate,Net Value Growth Rate,Interest Expense Ratio,Borrowing dependency,Net Worth Turnover Rate (times),Allocation rate per person,Inventory/Working Capital,Inventory/Current Liability,Retained Earnings to Total Assets,Cash Turnover Rate,Current Liability to Liability,Total assets to GNP price,Equity to Liability
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000,2.000000e+03,2000.000000
mean,0.603785,0.998843,0.781041,1.844045e+09,0.464422,3.732000e+07,0.631498,0.382959,0.039831,3.480000e+06,0.277368,3.585150e+07,0.919430,2.332955e+09,0.748211,7.206000e+07,0.034478
std,0.014686,0.000993,0.003344,3.190004e+09,0.012146,5.890469e+08,0.014577,0.052685,0.036178,1.100197e+08,0.005311,4.840476e+08,0.034846,2.687308e+09,0.204472,7.652163e+08,0.048426
min,0.449761,0.973424,0.735093,1.000256e-04,0.343818,0.000000e+00,0.525126,0.000000,0.009032,0.000000e+00,0.244753,0.000000e+00,0.728885,0.000000e+00,0.049582,1.015558e-04,0.003946
25%,0.597580,0.998857,0.781286,1.549529e-04,0.459606,3.896315e-04,0.630095,0.371918,0.021935,4.761171e-03,0.276920,2.487101e-03,0.909662,7.248945e-04,0.617815,9.988970e-04,0.017979
50%,0.602099,0.998976,0.781541,2.864894e-04,0.462599,4.406694e-04,0.630612,0.376805,0.029839,1.027135e-02,0.277135,5.344259e-03,0.930045,1.270000e+09,0.792352,2.623103e-03,0.023615
75%,0.609738,0.999044,0.781652,3.202500e+09,0.466664,4.735780e-04,0.631087,0.383465,0.043750,2.309766e-02,0.277511,9.879049e-03,0.938763,3.952500e+09,0.919076,6.552762e-03,0.035823
max,0.665151,0.999650,0.786079,9.980000e+09,0.617039,9.330000e+09,0.892562,1.000000,0.404839,3.480000e+09,0.466868,9.910000e+09,0.998858,1.000000e+10,1.000000,9.650000e+09,0.942729
